In [1]:
# 필요한 라이브러리 import
import ollama
from PIL import Image
from pathlib import Path
import time

print("✅ 라이브러리 import 완료")
print("⚠️  ollama 패키지가 없다면: pip install ollama")

✅ 라이브러리 import 완료
⚠️  ollama 패키지가 없다면: pip install ollama


## 1. GPU 환경 확인

In [2]:
print("=" * 70)
print("🔍 Ollama 연결 확인")
print("=" * 70)

# Ollama 서버 및 모델 확인
try:
    models = ollama.list()
    print(f"\n✓ Ollama 연결 성공")
    print(f"✓ 설치된 모델 수: {len(models['models'])}")
    
    # LLaVA 모델 확인
    llava_models = [m['name'] for m in models['models'] if 'llava' in m['name'].lower()]
    if llava_models:
        print(f"✓ LLaVA 모델 발견: {', '.join(llava_models)}")
    else:
        print("⚠️  LLaVA 모델이 설치되지 않았습니다!")
        print("   터미널에서 실행: ollama pull llava")
except Exception as e:
    print(f"❌ Ollama에 연결할 수 없습니다: {e}")
    print("   Ollama가 실행 중인지 확인하세요!")

print("\n" + "=" * 70)

🔍 Ollama 연결 확인

✓ Ollama 연결 성공
✓ 설치된 모델 수: 3
❌ Ollama에 연결할 수 없습니다: 'name'
   Ollama가 실행 중인지 확인하세요!



## 2. LLaVA 모델 로드

LLaVA-NeXT (llava-v1.6-mistral-7b) 사용

In [3]:
# 설정
TEST_IMAGE_DIR = Path("../test")  # 테스트 이미지 경로
MODEL_NAME = "llava:7b"
MAX_IMAGE_SIZE = 1024  # 이미지 최대 크기 (픽셀) - 더 높은 해상도로 세부사항 보존

print("=" * 70)
print("⚙️  설정")
print("=" * 70)
print(f"테스트 이미지 경로: {TEST_IMAGE_DIR.absolute()}")
print(f"모델: {MODEL_NAME}")
print(f"최대 이미지 크기: {MAX_IMAGE_SIZE}px")
print("=" * 70)

⚙️  설정
테스트 이미지 경로: c:\Users\helen\Desktop\kt cloud tech up\basic_project\models\captioning\..\test
모델: llava:7b
최대 이미지 크기: 1024px


## 3. 이미지 캡셔닝 함수 정의

In [4]:
def preprocess_image(image_path: Path, max_size: int = 1024) -> Path:
    """이미지 전처리 (크기 조정 및 품질 향상)"""
    with Image.open(image_path) as img:
        # RGB 변환
        if img.mode != 'RGB':
            img = img.convert('RGB')
        
        # 크기 조정 (품질 손실 최소화)
        if max(img.size) > max_size:
            img.thumbnail((max_size, max_size), Image.Resampling.LANCZOS)
            
            # 고품질로 임시 파일 저장
            temp_path = image_path.parent / f"temp_{image_path.name}"
            img.save(temp_path, format="JPEG", quality=95, optimize=False)
            return temp_path
    
    return image_path


def call_ollama(prompt: str, image_path: Path, temperature: float = 0.2) -> str:
    """Ollama API 호출 (ollama 라이브러리 사용)"""
    response = ollama.generate(
        model=MODEL_NAME,
        prompt=prompt,
        images=[str(image_path)],
        options={
            'temperature': temperature,  # 매우 낮은 temperature로 정확도 최대화
            'top_p': 0.95,
            'top_k': 50,
            'num_ctx': 4096  # 더 긴 컨텍스트로 이미지 정보 처리
        }
    )
    return response['response']


def validate_caption_with_verification(image_path: Path, caption: str, category: str) -> dict:
    """
    생성된 캡션을 검증하여 hallucination 감지
    
    Args:
        image_path: 이미지 경로
        caption: 생성된 캡션
        category: 이미지 카테고리 (TREE/HOUSE/PERSON)
        
    Returns:
        dict: {
            'is_valid': bool,
            'confidence': str,
            'issues': list,
            'verification_response': str
        }
    """
    print("      🔍 캡션 검증 중...")
    
    # 카테고리별 검증 질문
    if category == "TREE":
        verify_prompt = f"""Look at this tree drawing VERY CAREFULLY.

The following description was generated:
"{caption}"

Now answer these verification questions with ONLY yes/no/unclear:

1. Are branches clearly visible and drawn in this image? (yes/no/unclear)
2. If the description mentions branches, do you actually see branches? (yes/no/unclear/not-mentioned)
3. Are roots visible in this drawing? (yes/no/unclear)
4. If the description mentions roots, do you actually see roots? (yes/no/unclear/not-mentioned)
5. Is the crown/foliage dense and full, or sparse and minimal? (full/sparse/unclear)
6. Does the description accurately match what you see? (yes/no/partially)

Answer in this format:
1. [yes/no/unclear]
2. [yes/no/unclear/not-mentioned]
3. [yes/no/unclear]
4. [yes/no/unclear/not-mentioned]
5. [full/sparse/unclear]
6. [yes/no/partially]
If answer 6 is 'no' or 'partially', explain what's wrong."""

    elif category == "HOUSE":
        verify_prompt = f"""Look at this house drawing VERY CAREFULLY.

The following description was generated:
"{caption}"

Now answer these verification questions with ONLY yes/no/unclear:

1. Count the windows - how many do you see? (give exact number or say "unclear")
2. Does the description's window count match what you counted? (yes/no/unclear)
3. Is there a door in this drawing? (yes/no/unclear)
4. If the description mentions a door, do you actually see one? (yes/no/unclear/not-mentioned)
5. Is there a chimney? (yes/no/unclear)
6. Does the description accurately match what you see? (yes/no/partially)

Answer in this format:
1. [number or unclear]
2. [yes/no/unclear]
3. [yes/no/unclear]
4. [yes/no/unclear/not-mentioned]
5. [yes/no/unclear]
6. [yes/no/partially]
If answer 6 is 'no' or 'partially', explain what's wrong."""

    elif category == "PERSON":
        verify_prompt = f"""Look at this person drawing VERY CAREFULLY.

The following description was generated:
"{caption}"

Now answer these verification questions with ONLY yes/no/unclear:

1. Are facial features (eyes/nose/mouth) clearly drawn? (yes/no/unclear - specify which)
2. Does the description's facial feature details match what you see? (yes/no/unclear)
3. Can you count fingers on the hands? (yes/no/unclear - if yes, how many?)
4. If the description mentions finger count, is it accurate? (yes/no/unclear/not-mentioned)
5. Are feet/shoes visible and detailed? (yes/no/unclear)
6. Does the description accurately match what you see? (yes/no/partially)

Answer in this format:
1. [yes/no/unclear - details]
2. [yes/no/unclear]
3. [yes/no/unclear - count if yes]
4. [yes/no/unclear/not-mentioned]
5. [yes/no/unclear]
6. [yes/no/partially]
If answer 6 is 'no' or 'partially', explain what's wrong."""

    else:
        verify_prompt = f"""Look at this drawing VERY CAREFULLY.

The following description was generated:
"{caption}"

Does this description accurately match what you actually see in the image?
Answer: yes/no/partially
If no or partially, explain what's inaccurate."""

    # 검증 실행
    try:
        verification_response = call_ollama(verify_prompt, image_path, temperature=0.1)
        
        # 검증 결과 분석
        issues = []
        is_valid = True
        confidence = "HIGH"
        
        lower_response = verification_response.lower()
        
        # Hallucination 패턴 감지
        if "no" in lower_response or "inaccurate" in lower_response or "wrong" in lower_response:
            is_valid = False
            confidence = "LOW"
            issues.append("Description may not match the actual image")
            print("      ⚠️  HALLUCINATION 감지 가능성!")
        
        if "partially" in lower_response:
            confidence = "MEDIUM"
            issues.append("Description partially matches the image")
            print("      ⚠️  부분적 불일치 감지")
        
        if "unclear" in lower_response:
            confidence = "MEDIUM"
            issues.append("Some features are unclear in the image")
            print("      ⚠️  일부 특징이 불명확함")
        
        # 불일치 키워드 확인
        mismatch_keywords = ["not visible", "cannot see", "doesn't match", "does not match", "incorrect"]
        for keyword in mismatch_keywords:
            if keyword in lower_response:
                is_valid = False
                confidence = "LOW"
                issues.append(f"Verification detected: {keyword}")
                print(f"      ⚠️  불일치 감지: {keyword}")
        
        return {
            'is_valid': is_valid,
            'confidence': confidence,
            'issues': issues,
            'verification_response': verification_response
        }
        
    except Exception as e:
        print(f"      ❌ 검증 오류: {e}")
        return {
            'is_valid': True,  # 검증 실패 시 원본 캡션 유지
            'confidence': "UNKNOWN",
            'issues': [f"Verification failed: {str(e)}"],
            'verification_response': ""
        }

def generate_detailed_caption(image_path, max_size=512):
    """
    이미지에 대한 상세한 캡션을 생성합니다.
    
    Args:
        image_path: 이미지 파일 경로
        max_size: 이미지 최대 크기
        
    Returns:
        dict: category와 caption을 포함한 결과
    """
    temp_path = None
    try:
        start_time = time.time()
        
        # 이미지 전처리
        image_path = Path(image_path)
        processed_path = preprocess_image(image_path, max_size)
        if processed_path != image_path:
            temp_path = processed_path
        
        # 1단계: 카테고리 분류
        print(f"   🏷️  카테고리 분류 중...")
        
        category_prompt = """Look carefully at this drawing and identify the main subject.
This is a psychological test drawing (HTP - House-Tree-Person test).

What is the main subject of this drawing?
Answer with ONLY ONE word: HOUSE, PERSON, or TREE."""
        
        category_text = call_ollama(category_prompt, processed_path, temperature=0.1).strip().upper()
        
        # 카테고리 정규화
        if "HOUSE" in category_text or "HOME" in category_text or "BUILDING" in category_text:
            category = "HOUSE"
        elif "PERSON" in category_text or "PEOPLE" in category_text or "MAN" in category_text or "WOMAN" in category_text or "HUMAN" in category_text:
            category = "PERSON"
        elif "TREE" in category_text or "PLANT" in category_text:
            category = "TREE"
        else:
            category = "UNKNOWN"
        
        print(f"   📂 카테고리: {category}")
        
        # 2단계: 5가지 특성 설명 생성
        print(f"   📝 상세 설명 생성 중...")
        
        # 개선된 HTP 프롬프트 시스템
        if category == "TREE":
            description_prompt = """EXAMINE THIS TREE DRAWING WITH EXTREME CARE.

        This is a psychological assessment drawing (HTP test). Your description must be FACTUALLY ACCURATE.

        CRITICAL INSTRUCTIONS:
        1. Analyze the image carefully BEFORE writing anything
        2. If uncertain about ANY feature (branches, details, texture), write "cannot clearly determine" INSTEAD of guessing
        3. Write EXACTLY 5 sentences numbered 1-5
        4. Each sentence MUST start with "The tree"
        5. Describe ONLY what you ACTUALLY SEE - no general tree knowledge
        6. NO bullet points, dashes, or markdown formatting

        ACCURACY VERIFICATION - Answer these mentally FIRST:
        • Does this tree have visible branches? (yes/no/unclear) How many distinct areas show branches?
        • Is the crown full/moderately full/sparse? (pick ONE based on what you see)
        • Is the trunk marked? (smooth/with lines/with shading/cannot determine)
        • Are roots drawn? (yes/no/unclear)
        • Is any foliage/leaf texture visible? (yes/no)

        WHAT TO DESCRIBE (in this exact order):
        1. Crown appearance (size relative to trunk, shape - wide/narrow/pointed, fullness - full/sparse)
        2. Trunk characteristics (thickness, straightness, surface - smooth/textured/marked with what)
        3. Branches and twigs (are they visible? if yes - how many main branches, direction, thickness)
        4. Root system (visible or not? if visible - pattern and depth)
        5. Drawing technique (line weight - light/medium/heavy, shading presence, overall style)

        DO NOT:
        - Describe branches if you only see the trunk outline
        - Assume density or details not clearly visible
        - Use general tree descriptions
        - Guess about artistic intent

        IF YOU ARE UNSURE: Use "The tree's [feature] cannot be clearly determined from the image."

        NOW - Describe THIS SPECIFIC tree drawing:"""

        elif category == "HOUSE":
            description_prompt = """EXAMINE THIS HOUSE DRAWING WITH EXTREME CARE.

        This is a psychological assessment drawing (HTP test). Your description must be FACTUALLY ACCURATE.

        CRITICAL INSTRUCTIONS:
        1. Analyze the image carefully BEFORE writing anything
        2. If uncertain about ANY feature (count, details, texture), write "cannot clearly determine" INSTEAD of guessing
        3. Write EXACTLY 5 sentences numbered 1-5
        4. Each sentence MUST start with "The house"
        5. Describe ONLY what you ACTUALLY SEE - no assumptions
        6. NO bullet points, dashes, or markdown formatting

        ACCURACY VERIFICATION - Answer these mentally FIRST:
        • How many windows do you count? (exact number or "unclear")
        • Is there a door? (yes/no) If yes - where positioned? (center/left/right)
        • What roof style? (triangular/flat/curved/mixed)
        • Chimney present? (yes/no/unclear)
        • What is the base/foundation? (rectangular/other shape)

        WHAT TO DESCRIBE (in this exact order):
        1. Overall structure (size - large/small/medium, wall shape, base/foundation style)
        2. Roof (style - peaked/flat/curved, angle if peaked, chimney presence and position)
        3. Windows (exact count if visible, size, position - symmetric/asymmetric, details like frames/curtains)
        4. Door (present or absent? if present - position/size/details like handle/steps/path)
        5. Additional elements and technique (fence/path/smoke/surroundings, line quality - light/medium/heavy, shading)

        DO NOT:
        - Assume window or door count - count what you see
        - Describe details not visible in the drawing
        - Add typical house elements that aren't drawn
        - Guess about artistic intent

        IF YOU ARE UNSURE: Use "The house's [feature] cannot be clearly determined from the image."

        NOW - Describe THIS SPECIFIC house drawing:"""

        elif category == "PERSON":
            description_prompt = """EXAMINE THIS PERSON DRAWING WITH EXTREME CARE.

        This is a psychological assessment drawing (HTP test). Your description must be FACTUALLY ACCURATE.

        CRITICAL INSTRUCTIONS:
        1. Analyze the image carefully BEFORE writing anything
        2. If uncertain about ANY feature (proportions, details), write "cannot clearly determine" INSTEAD of guessing
        3. Write EXACTLY 5 sentences numbered 1-5
        4. Each sentence MUST start with "The person" or "The figure"
        5. Describe ONLY what you ACTUALLY SEE - no assumptions
        6. NO bullet points, dashes, or markdown formatting

        ACCURACY VERIFICATION - Answer these mentally FIRST:
        • What is the body proportion? (tall/short/medium relative to head size)
        • Posture? (standing/sitting/other)
        • Face details visible? (eyes/nose/mouth present? - yes/no for each)
        • Arms present? Fingers? (how many if countable)
        • Legs visible? Feet details? (shoes present? - yes/no)
        • Clothing? (shirt/pants/dress/other - describe what's actually drawn)

        WHAT TO DESCRIBE (in this exact order):
        1. Body structure (height relative to head, posture - standing/sitting/other, overall proportions)
        2. Head and face (size relative to body, facial features actually visible - which ones)
        3. Arms and hands (position, length, hand details - fingers visible and countable?)
        4. Legs and feet (stance, length, feet details - shoes drawn? yes/no)
        5. Clothing and style (what garments visible, hair visible, accessories, line quality - light/medium/heavy)

        DO NOT:
        - Assume facial features not clearly drawn
        - Describe typical proportions - describe THIS drawing's proportions
        - Add details that aren't visible
        - Guess about gender or expression if unclear

        IF YOU ARE UNSURE: Use "The person's [feature] cannot be clearly determined from the image."

        NOW - Describe THIS SPECIFIC person drawing:"""

        else:
            description_prompt = """Describe EXACTLY what you SEE in this drawing - nothing more, nothing less.

IMPORTANT RULES:
1. Write EXACTLY 5 sentences, numbered 1-5
2. Describe one specific visible feature per sentence
3. DO NOT speculate, assume, or add details not present
4. If you're uncertain about any detail, say "unclear" or "cannot determine"
5. NO bullet points, dashes, or markdown formatting
6. NO general statements - only describe THIS specific drawing

Before writing: Look at the drawing for 5 seconds and mentally note:
- What are the main elements?
- What details are clearly visible?
- What is NOT visible?
- Am I guessing about anything?

Now describe this drawing in 5 sentences:"""
        
        caption = call_ollama(description_prompt, processed_path).strip()
        
        # 캡션 검증 및 필요시 재생성 (최대 2회 시도)
        max_retries = 2
        retry_count = 0
        validation_result = None
        original_caption = caption
        all_attempts = [{"attempt": 1, "caption": caption}]
        
        while retry_count < max_retries:
            # 캡션 검증
            validation_result = validate_caption_with_verification(processed_path, caption, category)
            
            # 신뢰도가 HIGH이면 성공
            if validation_result['confidence'] == 'HIGH':
                print(f"   ✅ 신뢰도: {validation_result['confidence']} - 검증 통과!")
                break
            
            # 신뢰도가 LOW이면 재생성 시도
            if validation_result['confidence'] == 'LOW' and retry_count < max_retries - 1:
                retry_count += 1
                print(f"   🔄 낮은 신뢰도 감지 - 캡션 재생성 중... (시도 {retry_count + 1}/{max_retries})")
                print(f"   📋 검증 피드백: {', '.join(validation_result['issues'])}")
                
                # 검증 피드백을 포함한 개선된 프롬프트
                correction_prompt = f"""PREVIOUS ATTEMPT HAD ISSUES. Look at this {category.lower()} drawing again VERY CAREFULLY.

The previous description was:
"{caption}"

VERIFICATION FEEDBACK identified these problems:
{', '.join(validation_result['issues'])}

Verification details:
{validation_result['verification_response']}

NOW - Create a NEW, ACCURATE description that fixes these issues:

CRITICAL RULES (same as before):
1. Write EXACTLY 5 sentences numbered 1-5
2. Each sentence MUST start with "The {category.lower()}..."
3. ONLY describe what you ACTUALLY SEE - if you cannot see something clearly, say "cannot be clearly determined"
4. NO speculation, NO assumptions, NO typical {category.lower()} features unless actually visible
5. Count carefully (windows, fingers, branches, etc.)
6. If something is NOT visible, do NOT mention it

Double-check your answer before responding. Does each statement match what you actually see?

NOW - Write the corrected description:"""
                
                caption = call_ollama(correction_prompt, processed_path, temperature=0.15).strip()
                all_attempts.append({"attempt": retry_count + 1, "caption": caption})
                
            elif validation_result['confidence'] == 'MEDIUM':
                print(f"   ⚠️  신뢰도: MEDIUM - 일부 불명확한 요소 있음 (재생성 없이 계속)")
                break
            else:
                # LOW이지만 마지막 시도
                print(f"   ⚠️  신뢰도: {validation_result['confidence']} - 최대 재생성 횟수 도달")
                break
        
        elapsed = time.time() - start_time
        print(f"   ⏱️  처리 시간: {elapsed:.1f}초 (총 {len(all_attempts)}회 생성)")
        
        # 최종 검증 결과 출력
        if validation_result['confidence'] == 'LOW':
            print(f"   ❌ 최종 신뢰도: LOW - 수동 검증 필요!")
            print(f"   📋 이슈: {', '.join(validation_result['issues'])}")
        
        return {
            "category": category,
            "caption": caption,
            "validation": {
                "is_valid": validation_result['is_valid'],
                "confidence": validation_result['confidence'],
                "issues": validation_result['issues'],
                "verification_response": validation_result['verification_response'],
                "retry_count": retry_count,
                "all_attempts": all_attempts
            }
        }
        
    except Exception as e:
        print(f"❌ 오류 발생 ({image_path}): {e}")
        import traceback
        traceback.print_exc()
        return None
    finally:
        # 임시 파일 정리
        if temp_path and temp_path.exists():
            temp_path.unlink()


print("✅ 캡셔닝 함수 정의 완료")

✅ 캡셔닝 함수 정의 완료


## 4. 이미지 디렉토리 설정

In [5]:
# 이미지 디렉토리 설정
image_dir = r"C:\Users\helen\Desktop\kt cloud tech up\basic_project\test"
output_file = "image_captions_llava_with_validation2.json"

# 지원하는 이미지 확장자
image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.gif', '.webp']

# 이미지 파일 찾기
image_files = []
for ext in image_extensions:
    image_files.extend(Path(image_dir).glob(f'*{ext}'))
    image_files.extend(Path(image_dir).glob(f'*{ext.upper()}'))

image_files = sorted(set(image_files))

print(f"📁 이미지 디렉토리: {image_dir}")
print(f"🖼️  발견된 이미지: {len(image_files)}개\n")

if len(image_files) == 0:
    print("⚠️  이미지 파일이 없습니다!")
else:
    for img in image_files:
        print(f"   - {img.name}")

📁 이미지 디렉토리: C:\Users\helen\Desktop\kt cloud tech up\basic_project\test
🖼️  발견된 이미지: 4개

   - test_나무.JPG
   - test_남자사람.JPG
   - test_여자사람.JPG
   - test_집.JPG


## 5. 캡션 생성 실행

In [6]:
# 캡션 생성
results = {}

print("\n" + "=" * 70)
print("🎨 캡션 생성 시작")
print("=" * 70 + "\n")

total_start_time = time.time()

for i, image_path in enumerate(image_files, 1):
    print(f"\n[{i}/{len(image_files)}] 처리 중: {image_path.name}")
    
    result = generate_detailed_caption(str(image_path))
    
    if result:
        results[image_path.name] = {
            "path": str(image_path),
            "category": result["category"],
            "caption": result["caption"],
            "validation": result.get("validation", {})
        }
        print(f"✅ 완료")
        print(f"📂 카테고리: {result['category']}")
        print(f"📝 캡션: {result['caption'][:200]}{'...' if len(result['caption']) > 200 else ''}")
        
        # 검증 결과 요약 출력
        validation = result.get("validation", {})
        retry_count = validation.get('retry_count', 0)
        if retry_count > 0:
            print(f"🔄 캡션 재생성: {retry_count}회")
        
        if validation.get('confidence') == 'LOW':
            print(f"⚠️  경고: 낮은 신뢰도 - 수동 검증 필요!")
        elif validation.get('confidence') == 'MEDIUM':
            print(f"ℹ️  정보: 중간 신뢰도 - 일부 불명확한 요소 있음")
        elif validation.get('confidence') == 'HIGH':
            print(f"✅ 높은 신뢰도 - 정확한 캡션")
    else:
        print(f"⚠️  캡션 생성 실패")
    
    # 약간의 대기 (Ollama 서버 부하 방지)
    time.sleep(0.5)

total_elapsed = time.time() - total_start_time

print("\n" + "=" * 70)
print(f"✨ 총 {len(results)}개 이미지 처리 완료!")
print(f"⏱️  전체 소요 시간: {total_elapsed:.1f}초 (평균: {total_elapsed/len(image_files):.1f}초/이미지)")
print("=" * 70)


🎨 캡션 생성 시작


[1/4] 처리 중: test_나무.JPG
   🏷️  카테고리 분류 중...
   📂 카테고리: TREE
   📝 상세 설명 생성 중...
   📂 카테고리: TREE
   📝 상세 설명 생성 중...
      🔍 캡션 검증 중...
      🔍 캡션 검증 중...
      ⚠️  HALLUCINATION 감지 가능성!
   🔄 낮은 신뢰도 감지 - 캡션 재생성 중... (시도 2/2)
   📋 검증 피드백: Description may not match the actual image
      ⚠️  HALLUCINATION 감지 가능성!
   🔄 낮은 신뢰도 감지 - 캡션 재생성 중... (시도 2/2)
   📋 검증 피드백: Description may not match the actual image
      🔍 캡션 검증 중...
      🔍 캡션 검증 중...
      ⚠️  HALLUCINATION 감지 가능성!
      ⚠️  불일치 감지: not visible
   ⚠️  신뢰도: LOW - 최대 재생성 횟수 도달
   ⏱️  처리 시간: 52.9초 (총 2회 생성)
   ❌ 최종 신뢰도: LOW - 수동 검증 필요!
   📋 이슈: Description may not match the actual image, Verification detected: not visible
✅ 완료
📂 카테고리: TREE
📝 캡션: 1. The tree has a sparse and narrow crown with no distinct branches or twigs.
2. The trunk of the tree is straight and smooth, with no visible markings or shading.
3. The roots of the tree are not vis...
🔄 캡션 재생성: 1회
⚠️  경고: 낮은 신뢰도 - 수동 검증 필요!
      ⚠️  HALLUCINATION 감지 가능성!
    

## 6. 결과 저장

In [7]:
# JSON 파일로 저장
import json

with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

print(f"\n💾 결과 저장 완료: {output_file}")
print(f"📊 총 {len(results)}개 이미지 캡션 저장됨\n")

# 검증 결과 통계
print("=" * 70)
print("📊 검증 결과 통계")
print("=" * 70)
high_conf = sum(1 for r in results.values() if r.get('validation', {}).get('confidence') == 'HIGH')
medium_conf = sum(1 for r in results.values() if r.get('validation', {}).get('confidence') == 'MEDIUM')
low_conf = sum(1 for r in results.values() if r.get('validation', {}).get('confidence') == 'LOW')
retry_count = sum(r.get('validation', {}).get('retry_count', 0) for r in results.values())
print(f"✅ 높은 신뢰도: {high_conf}개")
print(f"⚠️  중간 신뢰도: {medium_conf}개")
print(f"❌ 낮은 신뢰도: {low_conf}개 (수동 검증 필요!)")
print(f"🔄 총 재생성 횟수: {retry_count}회")

# 결과 미리보기
print("\n" + "=" * 70)
print("📋 생성된 캡션 요약")
print("=" * 70)
for filename, data in results.items():
    print(f"\n🖼️  {filename}")
    print(f"   📂 카테고리: {data.get('category', 'UNKNOWN')}")
    validation = data.get('validation', {})
    confidence_emoji = "✅" if validation.get('confidence') == 'HIGH' else ("⚠️" if validation.get('confidence') == 'MEDIUM' else "❌")
    print(f"   {confidence_emoji} 신뢰도: {validation.get('confidence', 'UNKNOWN')}")
    
    retry_count = validation.get('retry_count', 0)
    if retry_count > 0:
        print(f"   🔄 재생성: {retry_count}회 (총 {len(validation.get('all_attempts', []))}회 시도)")
    
    if validation.get('issues'):
        print(f"   ⚠️  이슈: {', '.join(validation['issues'])}")
    print(f"   📝 캡션: {data['caption'][:150]}{'...' if len(data['caption']) > 150 else ''}")


💾 결과 저장 완료: image_captions_llava_with_validation2.json
📊 총 4개 이미지 캡션 저장됨

📊 검증 결과 통계
✅ 높은 신뢰도: 0개
⚠️  중간 신뢰도: 0개
❌ 낮은 신뢰도: 4개 (수동 검증 필요!)
🔄 총 재생성 횟수: 4회

📋 생성된 캡션 요약

🖼️  test_나무.JPG
   📂 카테고리: TREE
   ❌ 신뢰도: LOW
   🔄 재생성: 1회 (총 2회 시도)
   ⚠️  이슈: Description may not match the actual image, Verification detected: not visible
   📝 캡션: 1. The tree has a sparse and narrow crown with no distinct branches or twigs.
2. The trunk of the tree is straight and smooth, with no visible marking...

🖼️  test_남자사람.JPG
   📂 카테고리: PERSON
   ❌ 신뢰도: LOW
   🔄 재생성: 1회 (총 2회 시도)
   ⚠️  이슈: Description may not match the actual image
   📝 캡션: 1. The person appears to be standing upright with a medium height relative to their head size.
2. The head of the figure is proportionate to the body,...

🖼️  test_여자사람.JPG
   📂 카테고리: PERSON
   ❌ 신뢰도: LOW
   🔄 재생성: 1회 (총 2회 시도)
   ⚠️  이슈: Description may not match the actual image, Verification detected: not visible
   📝 캡션: 1. The person is standing upright with a clear